# Results from cruise with F/F Trygve Braarud 28th September 2023

## Drifters

In [1]:
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic as GD
import warnings

import matplotlib.pyplot as plt
import numpy as np
from cartopy import crs as ccrs, feature as cfeature
import cartopy

import re

# Suppress warnings issued by Cartopy when downloading data files
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("C:\\Users\\stang\\GEO2320\\Cruise data\\drifters\\UiO GEO 2320 sept 2023.txt", delimiter=r"\t+")
df

,Layer,Icon,Color,Name,Position,Comment,Modified,Depth,Locked
0,UiO GEO 2320 2023,0,Red,Flyterigg inn 0m 14:23,N 59°52.373'; E 010°41.729',,"15:11:51 Thursday, September 28, 2023",71.0 m,False
1,UiO GEO 2320 2023,0,Red,Flyterigg inn 20m 14:35,N 59°52.109'; E 010°41.968',,"15:11:57 Thursday, September 28, 2023",75.7 m,False
2,UiO GEO 2320 2023,0,Red,Flyterigg inn 3m 14:27,N 59°52.255'; E 010°41.865',,"15:12:00 Thursday, September 28, 2023",72.1 m,False
3,UiO GEO 2320 2023,0,Red,Flyterigg inn 5m 14:30,N 59°52.252'; E 010°41.892',,"15:12:03 Thursday, September 28, 2023",70.6 m,False
4,UiO GEO 2320 2023,0,Magenta,Flytrerigg 2 satt 13:47,N 59°52.133'; E 010°42.017',,"15:12:11 Thursday, September 28, 2023",75.4 m,False
5,UiO GEO 2320 2023,0,LightGreen,BN-1 Lysaker fj.,N 59°52.838'; E 010°38.805',,"15:12:49 Thursday, September 28, 2023",84.6 m,False
6,UiO GEO 2320 2023,0,LightGreen,CP1- Oksenvallflua,N 59°51.853'; E 010°42.083',,"15:14:35 Thursday, September 28, 2023",85.6 m,False
7,UiO GEO 2320 2023,0,LightGreen,R2 Terskel,N 59°53.199'; E 010°39.965',,"15:14:13 Thursday, September 28, 2023",52.5 m,False
8,UiO GEO 2320 2023,0,Yellow,Flyterigg inn 0m 11:58,N 59°51.741'; E 010°40.945',,"15:11:50 Thursday, September 28, 2023",26.2 m,False
9,UiO GEO 2320 2023,0,Yellow,Flyterigg inn 20m 11:51,N 59°51.952'; E 010°41.793',,"15:11:53 Thursday, September 28, 2023",54.2 m,False


In [3]:
# Function for converting coordinates
def coordinates(position):
    lat = position[0]
    lon = position[1]

    deg, minutes = re.split('[°"]', lat)
    lat = (float(deg) + float(minutes)/60 )

    deg, minutes = re.split('[°"]', lon)
    lon = (float(deg) + float(minutes)/60 )

    return (lat, lon)

# Function for calculating distance between two coordinate points
def distance(lat1, lon1, lat2, lon2):
    start = (lat1, lon1)
    end = (lat2, lon2)
    distance = GD(start, end).m
    return distance

In [4]:
# Tokt 1
# Extracting the coordinates
position_start_1 = str(df["Position"][12]).split("; ")
position_start_1 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_start_1]

position_end_0_1 = str(df["Position"][8]).split("; ")
position_end_0_1 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_end_0_1]

position_end_3_1 = str(df["Position"][10]).split("; ")
position_end_3_1 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_end_3_1]

position_end_5_1 = str(df["Position"][11]).split("; ")
position_end_5_1 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_end_5_1]

position_end_20_1 = str(df["Position"][9]).split("; ")
position_end_20_1 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_end_20_1]

# Converting to decimal cooridantes

position_start_1 = coordinates(position_start_1)
position_end_0_1 = coordinates(position_end_0_1)
position_end_3_1 = coordinates(position_end_3_1)
position_end_5_1 = coordinates(position_end_5_1)
position_end_20_1 = coordinates(position_end_20_1)

# Extracting the times of deployment
time_start_1 = "09:52"
time_start_1 = datetime.strptime(time_start_1, "%H:%M")

# List of pick-up times of the drifters, drouges at 0, 3, 5, 20 meters respectively
time_end_1 = ["11:58", "11:42", "11:45", "11:51"]

for i in range (4):
    time_end_1[i] = datetime.strptime(time_end_1[i],"%H:%M")

# List to store the time each drifter had to drift away [seconds]
time_elapsed_1 = []

for i in range(4):
    time_drifting = time_end_1[i] - time_start_1
    time_elapsed_1.append(time_drifting.total_seconds())


# Retrieving the distances each drifter drifted
distance_0_1 = distance(position_start_1[0], position_start_1[1], position_end_0_1[0], position_end_0_1[1])
distance_3_1 = distance(position_start_1[0], position_start_1[1], position_end_3_1[0], position_end_3_1[1])
distance_5_1 = distance(position_start_1[0], position_start_1[1], position_end_5_1[0], position_end_5_1[1])
distance_20_1= distance(position_start_1[0], position_start_1[1], position_end_20_1[0], position_end_20_1[1])

# Finding the mean velocity of each drifter, here in [m/s]
vel_0_1 = distance_0_1/time_elapsed_1[0]
vel_3_1 = distance_3_1/time_elapsed_1[1]
vel_5_1 = distance_5_1/time_elapsed_1[2]
vel_20_1 = distance_20_1/time_elapsed_1[3]


In [5]:
print("Here are the drifter results for leg 1: \n")
print(f"The starting position of all drifters were latitide: {position_start_1} \n")
print()
print(f"The drifter with sail at 0 m travelled for {time_elapsed_1[0]} approximate seconds, and ended up at {position_end_0_1}. \n")
print(f"The drifter travelled {distance_0_1} meters.")
print(f"The mean velocity for this drifter was {vel_0_1} m/s.")
print()
print()
print(f"The drifter with sail at 3 m travelled for {time_elapsed_1[1]} approximate seconds, and ended up at {position_end_3_1}. \n")
print(f"The drifter travelled {distance_3_1} meters.")
print(f"The mean velocity for this drifter was {vel_3_1} m/s.")
print()
print()
print(f"The drifter with sail at 5 m travelled for {time_elapsed_1[2]} approximate seconds, and ended up at {position_end_5_1}. \n")
print(f"The drifter travelled {distance_5_1} meters.")
print(f"The mean velocity for this drifter was {vel_5_1} m/s.")
print()
print()
print(f"The drifter with sail at 20 m travelled for {time_elapsed_1[3]} approximate seconds, and ended up at {position_end_20_1}. \n")
print(f"The drifter travelled {distance_20_1} meters.")
print(f"The mean velocity for this drifter was {vel_20_1} m/s.")

Here are the drifter results for leg 1: 

The starting position of all drifters were latitide: (59.8685, 10.700283333333333) 


The drifter with sail at 0 m travelled for 7560.0 approximate seconds, and ended up at (59.86235, 10.682416666666667). 

The drifter travelled 1213.0430562098052 meters.
The mean velocity for this drifter was 0.1604554307155827 m/s.


The drifter with sail at 3 m travelled for 6600.0 approximate seconds, and ended up at (59.866933333333336, 10.69375). 

The drifter travelled 405.50166622694167 meters.
The mean velocity for this drifter was 0.061439646398021464 m/s.


The drifter with sail at 5 m travelled for 6780.0 approximate seconds, and ended up at (59.86763333333333, 10.695616666666666). 

The drifter travelled 278.69658523808863 meters.
The mean velocity for this drifter was 0.04110569103806617 m/s.


The drifter with sail at 20 m travelled for 7140.0 approximate seconds, and ended up at (59.86586666666667, 10.69655). 

The drifter travelled 360.30161374

In [6]:
# Tokt 2
# Extracting the coordinates
position_start_2 = str(df["Position"][4]).split("; ")
position_start_2 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_start_2]

position_end_0_2 = str(df["Position"][0]).split("; ")
position_end_0_2 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_end_0_2]

position_end_3_2 = str(df["Position"][2]).split("; ")
position_end_3_2 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_end_3_2]

position_end_5_2 = str(df["Position"][3]).split("; ")
position_end_5_2 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_end_5_2]

position_end_20_2 = str(df["Position"][1]).split("; ")
position_end_20_2 = [coord.replace("N", "").replace("E", "").replace("'", "") for coord in position_end_20_2]

# Converting to decimal cooridantes

position_start_2 = coordinates(position_start_2)
position_end_0_2 = coordinates(position_end_0_2)
position_end_3_2 = coordinates(position_end_3_2)
position_end_5_2 = coordinates(position_end_5_2)
position_end_20_2 = coordinates(position_end_20_2)

# Extracting the times of deployment
time_start_2 = "13:47"
time_start_2 = datetime.strptime(time_start_2, "%H:%M")

# List of pick-up times of the drifters, drouges at 0, 3, 5, 20 meters respectively
time_end_2 = ["14:23", "14:27", "14:30", "14:35"]

for i in range (4):
    time_end_2[i] = datetime.strptime(time_end_2[i],"%H:%M")

# List to store the time each drifter had to drift away [seconds]
time_elapsed_2 = []

for i in range(4):
    time_drifting = time_end_2[i] - time_start_2
    time_elapsed_2.append(time_drifting.total_seconds())


# Retrieving the distances each drifter drifted
distance_0_2 = distance(position_start_2[0], position_start_2[1], position_end_0_2[0], position_end_0_2[1])
distance_3_2 = distance(position_start_2[0], position_start_2[1], position_end_3_2[0], position_end_3_2[1])
distance_5_2 = distance(position_start_2[0], position_start_2[1], position_end_5_2[0], position_end_5_2[1])
distance_20_2 = distance(position_start_2[0], position_start_2[1], position_end_20_2[0], position_end_20_2[1])

# Finding the mean velocity of each drifter, here in [m/s]
vel_0_2 = distance_0_2/time_elapsed_2[0]
vel_3_2 = distance_3_2/time_elapsed_2[1]
vel_5_2 = distance_5_2/time_elapsed_2[2]
vel_20_2 = distance_20_2/time_elapsed_2[3]

In [7]:
print("Here are the drifter results for leg 2: \n")
print(f"The starting position of all drifters were latitide: {position_start_2} \n")
print()
print(f"The drifter with sail at 0 m travelled for {time_elapsed_2[0]} approximate seconds, and ended up at {position_end_0_2}. \n")
print(f"The drifter travelled {distance_0_2} meters.")
print(f"The mean velocity for this drifter was {vel_0_2} m/s.")
print()
print()
print(f"The drifter with sail at 3 m travelled for {time_elapsed_2[1]} approximate seconds, and ended up at {position_end_3_2}. \n")
print(f"The drifter travelled {distance_3_2} meters.")
print(f"The mean velocity for this drifter was {vel_3_2} m/s.")
print()
print()
print(f"The drifter with sail at 5 m travelled for {time_elapsed_2[2]} approximate seconds, and ended up at {position_end_5_2}. \n")
print(f"The drifter travelled {distance_5_2} meters.")
print(f"The mean velocity for this drifter was {vel_5_2} m/s.")
print()
print()
print(f"The drifter with sail at 20 m travelled for {time_elapsed_2[3]} approximate seconds, and ended up at {position_end_20_2}. \n")
print(f"The drifter travelled {distance_20_2} meters.")
print(f"The mean velocity for this drifter was {vel_20_2} m/s.")

Here are the drifter results for leg 2: 

The starting position of all drifters were latitide: (59.868883333333336, 10.700283333333333) 


The drifter with sail at 0 m travelled for 2160.0 approximate seconds, and ended up at (59.872883333333334, 10.695483333333334). 

The drifter travelled 520.4742009835779 meters.
The mean velocity for this drifter was 0.24096027823313793 m/s.


The drifter with sail at 3 m travelled for 2400.0 approximate seconds, and ended up at (59.870916666666666, 10.69775). 

The drifter travelled 267.31505694150525 meters.
The mean velocity for this drifter was 0.11138127372562719 m/s.


The drifter with sail at 5 m travelled for 2580.0 approximate seconds, and ended up at (59.870866666666664, 10.6982). 

The drifter travelled 249.8902295593388 meters.
The mean velocity for this drifter was 0.09685667812377473 m/s.


The drifter with sail at 20 m travelled for 2880.0 approximate seconds, and ended up at (59.86848333333333, 10.699466666666666). 

The drifter tra